In [1]:
#Import Library
import os
import shutil
import glob
import cv2

In [2]:
#Drive Mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#GPU 
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [4]:
#데이터를 배열로 초기화
data = {}
data['crack'] = []
data['uncrack'] = []

In [5]:
#Crack Data
src = '/content/drive/MyDrive/딥러닝 오소리 /Data Set/10000 DataSet/Positive'
for jpgImage in glob.iglob(os.path.join(src, '*jpg')):
    data['crack'].append(jpgImage)
print(len(data['crack']))

10000


In [6]:
#Uncrack Data
src = '/content/drive/MyDrive/딥러닝 오소리 /Data Set/10000 DataSet/Negative'
for jpgImage in glob.iglob(os.path.join(src, '*jpg')):
    data['uncrack'].append(jpgImage)
print(len(data['uncrack']))

10000


In [7]:
#훈련에 필요한 파일 생성
os.mkdir('master_data_concrete')
os.mkdir('master_data_concrete/training')
os.mkdir('master_data_concrete/validation')
os.mkdir('master_data_concrete/test')

os.mkdir('master_data_concrete/training/crack')
os.mkdir('master_data_concrete/training/uncrack')
os.mkdir('master_data_concrete/validation/crack')
os.mkdir('master_data_concrete/validation/uncrack')
os.mkdir('master_data_concrete/test/crack')
os.mkdir('master_data_concrete/test/uncrack')

In [21]:
import random
split_size = 0.60
for class_type, imagesList in data.items():
    train_size = int(split_size*(len(imagesList)))
    train_images_list = random.shuffle(train_size)
print(train_size)

TypeError: ignored

In [18]:
#60% = training data // 40% = validation data 각각 이미지 파일 복사
#이 과정은 직접 이미지 파일을 사전에 복사 함 으로써 해결가능 그러나 그렇게 되면 능동적인 활용은 안된다.
import random
split_size = 0.60
for class_type, imagesList in data.items():
    train_size = int(split_size*(len(imagesList)))
    train_images_list = imagesList(random.shuffle(train_size))
    validation_images_list = imagesList[train_size:]
    
    base_dest = 'master_data_concrete'
    #copy training  images of crack type
    for image in train_images_list:
        dest = os.path.join(base_dest, 'training', class_type)
        shutil.copy(image, dest)

    #copy testing images of crack type
    for image in validation_images_list:
        dest = os.path.join(base_dest, 'validation', class_type)
        shutil.copy(image, dest)

TypeError: ignored

In [ ]:
#test  6:2:2 의 비율로 
train_size = 0.50
for class_type, imagesList in data.items():
    validation_size = int(train_size*(len(imagesList)))
    validation_images_list = imagesList[:validation_size]
    test_images_list = imagesList[validation_size:]
    
    #copy testing images of crack type
    for image in test_images_list:
        dest = os.path.join(base_dest, 'test', class_type)
        shutil.copy(image, dest)

In [ ]:
#Model Library
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D,BatchNormalization
from keras.callbacks import EarlyStopping, LambdaCallback
from keras.layers import Activation, Dropout,Input, Flatten, Dense
from keras.applications import ResNet50
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
#Convolutional Layer
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(227, 227, 3)))
model.add(BatchNormalization(axis=-1)) #배치 정규화
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [ ]:
#Output Layer
model.add(Flatten())  #3D를 1D로 벡터 변환
model.add(Dense(256))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1)) 
model.add(BatchNormalization(axis=-1))#this ===== 
model.add(Activation('sigmoid'))  #or model.add(Activation('softmax')) // two output softmax == one output sigmoid

In [ ]:
model.summary()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

batch_size = 30
epochs = 20

#스케일 조정 및 과적합 방지
train_datagen = ImageDataGenerator(
        rescale=1./255, 
        shear_range=0.2,
        #gray_range = cv2.cvtColor(temp, cv2.COLOR_BGR2GRAY)  #gray scale
        zoom_range=0.2,
        horizontal_flip=True)

#validation에 사용할 확장구성
validation_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)
#테스트, 검증데이터의 모든 이미지 크기 = 227*227
train_generator = train_datagen.flow_from_directory(
        '/content/master_data_concrete/training',  
        target_size=(227, 227), 
        batch_size=batch_size,
        #그레이스케일 추가 2
        class_mode='binary')   #binary_crossentropy loss를 사용하기 위한 binary label

#검증 데이터
validation_generator = validation_datagen.flow_from_directory(
        '/content/master_data_concrete/validation',
        target_size=(227, 227),
        batch_size=batch_size,
        class_mode='binary')

#test_generator = test_datagen.flow_from_directory(
 #       '/content/master_data_concrete/test',
  #      target_size=(227, 227),
   #     batch_size=batch_size,
    #    class_mode='binary')

Found 12000 images belonging to 2 classes.
Found 8000 images belonging to 2 classes.
Found 10000 images belonging to 2 classes.


In [ ]:
#만약 overfitting, underfitting 을 일으키면 EarlyStopping을 쓰기 위한 코드
#patience가 5면 validation score가 5회 이상 개선이 없을때 훈련을 멈춘다

#from keras.callbacks import EarlyStopping
#es = EarlyStopping(monitor='val_accuracy', patience=5, min_delta=0.005)

In [ ]:
#Model Learning
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) #rmsprop or adam , accuracy or f1-score  (positive와 negative의 데이터 값 차이가 높을 경우)
history = model.fit_generator(
        train_generator,
        epochs=epochs,
        validation_data=validation_generator) #callbacks=[es])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
400/400 [==============================] - 176s 357ms/step - loss: 0.3388 - accuracy: 0.9621 - val_loss: 0.4606 - val_accuracy: 0.7613
Epoch 2/20
400/400 [==============================] - 142s 355ms/step - loss: 0.2340 - accuracy: 0.9902 - val_loss: 0.1869 - val_accuracy: 0.9825
Epoch 3/20
400/400 [==============================] - ETA: 0s - loss: 0.1802 - accuracy: 0.9917

In [ ]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmp_xeh9uff/assets


In [ ]:
#훈련 및 검증 정확도 시각화
#학습 정확성 값과 검증 정확성 값
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'validation'], loc='upper left')
plt.show()

# 학습 손실 값과 검증 손실 값
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'validation'], loc='upper left')
plt.show()

In [ ]:
model.evaluate_generator(validation_generator)